In [2]:
import pandas as pd
import os

# folder locations
folders = [
    "C:/Users/Nila/Documents/after_shorten_final/BART-CNN/depth_first",
    "C:/Users/Nila/Documents/after_shorten_final/BART-CNN/divide_conquer",
    "C:/Users/Nila/Documents/after_shorten_final/GPT-4/depth_first",
    "C:/Users/Nila/Documents/after_shorten_final/GPT-4/divide_conquer",
    "C:/Users/Nila/Documents/after_shorten_final/GPT-4/json",
    "C:/Users/Nila/Documents/after_shorten_final/LLAMA2/depth_first",
    "C:/Users/Nila/Documents/after_shorten_final/LLAMA2/divide_conquer",
    "C:/Users/Nila/Documents/after_shorten_final/LLAMA2/json",
]

arg_folder = "C:/Users/Nila/Documents/arg_graphs"

# empty DataFrame
df = pd.DataFrame(columns=["Model 1", "Approach 1", "Model 2", "Approach 2", "Filename", "Summary 1", "Summary 2"])

# Looping through each file in the first folder
first_folder = folders[0]
df_list = []

for filename in os.listdir(first_folder):
    
    #lists to store information for each summary
    model_list = []
    approach_list = []
    summaries = []
    source_arg = []

    # Read information and summaries for each filename from all folders
    for folder in folders:
        folder_split = folder.split("/")
        model = folder_split[-2]
        approach = folder_split[-1]
        model_list.append(model)
        approach_list.append(approach)

        file_path = os.path.join(folder, filename)
        #print(file_path)

        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
            summaries.append(content)
      
    # Generate pairs for the summaries
    # only unique pairs will be created
    for i in range(len(summaries)):
        for j in range(i + 1, len(summaries)):
            model1 = model_list[i]
            approach1 = approach_list[i]
            model2 = model_list[j]
            approach2 = approach_list[j]

            summary1 = summaries[i]
            summary2 = summaries[j]
            
            # Removing the extension and split by hyphen
            filename_without_extension = os.path.splitext(filename)[0]
            parts = filename_without_extension.split('-')
            updated_filename = ' '.join(parts[:-1])
            question = f'Consider the summaries related to the topic "{updated_filename}". Please choose the summary that you think has the best "overall quality". Factors such as relevance, coherence, and fluency can be considered when making your choice.'

            # Add the argument content.
            arg_file_path = os.path.join(arg_folder, filename)

            with open(arg_file_path, "r", encoding="utf-8") as arg_file:
                arg_content = arg_file.read()

            #get the filename without extension
            filename_without_extension = os.path.splitext(filename)[0]
            arg_link = "www.kialo.com/" + filename_without_extension

            # Create a DataFrame
            pair_df = pd.DataFrame({
                "ArgGraph": [arg_link],
                "Model 1": [model1],
                "Approach 1": [approach1],
                "Model 2": [model2],
                "Approach 2": [approach2],
                "Question": [question],
                "Summary 1": [summary1],
                "Summary 2": [summary2],
                "Source": [arg_content],
            })


            # Append the DataFrame to the list
            df_list.append(pair_df)

            # Concatenate all DataFrames in the list
            final_df = pd.concat(df_list, ignore_index=True)
            # write dataframe to excel
            final_df.to_excel("all_questions_8feb16.xlsx", index=False)

In [3]:
pip install openpyxl


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: openpyxl in c:\users\nila\appdata\local\programs\python\python311\lib\site-packages (3.1.2)




[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd

questions_df = pd.read_excel('all_questions_8feb16.xlsx')

# Shuffle the rows
shuffle_df = questions_df.sample(frac=1.0, random_state=52)

shuffle_df.insert(0, 'Index', range(1, len(shuffle_df) + 1))

shuffle_df.to_excel('shuffled_file_8feb16.xlsx', index=False)

In [7]:
# split the shuffled file into multiple batches

import pandas as pd
import math

df = pd.read_excel('shuffled_file_8feb16.xlsx')

# Calculate the number of splits needed
# round off
total_record = 1400
chunk_size = 250
num_splits = math.ceil(total_record / chunk_size)

# Split the initial data into chunks of 250 records each and write them to separate files
for i in range(num_splits):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, total_record)
    
    # Create a subset
    subset = df.iloc[start_idx:end_idx]
    
    output_file = f'summaries_batch{i + 1}.xlsx'

    output_folder = f'./batches/{output_file}'
    
    # Write the result
    subset.to_excel(output_folder, index=False, header=True)